In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import SVG, display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# Untitled

**PHYS 395 project 2; **
**Matt Wiens - #301294492**

## Notebook setup 

The first command here sets the default figure size to be a bit larger than normal. The second command sets it so all figure output areas are expanded by default.

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (11, 11)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Introduction

Write introduction here

# Methods

Write methods here

# Analysis

## Illustrating fractal behaviour

**ignore**

To get a start at understanding how randomness can generate fractals, we'll look at a few examples.

### The Sierpiński triangle

**ignore**

Here we'll consider how we can randomly generate the Sierpiński triangle. Classically, you can generate this object by starting with any equilateral triangle and subdividing this triangle it into four smaller equilateral triangles of the same size (this subdivision is unique). After removing the "central" triangle, apply a similar subdivision to each of the remaining triangles, and repeat this process ad infinitum. Wikipedia has a good illustration of the end result [here][siepgen]).

Surprisingly, we can also approximate this triangle 
 
[siepgen]: https://en.wikipedia.org/wiki/Sierpi%C5%84ski_triangle#Removing_triangles

In [ ]:
svg_url = "https://upload.wikimedia.org/wikipedia/commons/0/05/Sierpinski_triangle_evolution.svg"
display(SVG(url=svg_url))

In [ ]:
# Vertices of the equilateral triangle
vertices = np.array([[0.0, 0.0], [0.5, 1.0], [1.0, 0.0]])

# Pick a random point within the triangle to start
s, t = np.sort(np.random.random(2))
init_point = np.array(
    [
        s * vertices[0, 0] + (t - s) * vertices[1, 0] + (1 - t) * vertices[2, 0],
        s * vertices[0, 1] + (t - s) * vertices[1, 1] + (1 - t) * vertices[2, 1],
    ]
)

In [ ]:
# Plot the initial point and the vertices
_, ax = plt.subplots()

ax.scatter(x=vertices[:, 0], y=vertices[:, 1], c="k", marker="*")
ax.scatter(x=init_point[0], y=init_point[1], c="r", marker="o")

ax.legend(["vertices", "initial point"]);

In [ ]:
# Set up an array with all the points we'll compute
num_points = 25000

points = np.zeros((num_points, 2))
points[0, :] = init_point

In [ ]:
# Perform the simulation
for i in range(1, num_points):
    r = np.random.randint(3)
    
    points[i, :] = 0.5 * (points[i - 1, :] + vertices[r, :])

In [ ]:
# Plot simulation results
_, ax = plt.subplots()

ax.scatter(x=points[:, 0], y=points[:, 1], s=0.5);

### Fern

In [ ]:
# Set up array to store data
num_points = 25000

points = np.zeros((num_points, 3))

# Initial point is fixed
points[0, :] = np.array([0.5, 0.0, -0.2])

In [ ]:
# Setup mode transformation matrices
mode_1_mat = np.array([[0.0, 0.0, 0.0], [0.0, 0.18, 0.0], [0.0, 0.0, 0.0]])
mode_2_mat = np.array([[0.85, 0.0, 0.0], [0.0, 0.85, 0.1], [0.0, -0.1, 0.85]])
mode_3_mat = np.array([[0.2, -0.2, 0.0], [0.2, 0.2, 0.0], [0.0, 0.0, 0.3]])
mode_4_mat = np.array([[-0.2, 0.2, 0.0], [0.2, 0.2, 0.0], [0.0, 0.0, 0.3]])

# Constants to add after each matrix product
mode_1_consts = np.array([0.0, 0.0, 0.0])
mode_2_consts = np.array([0.0, 1.6, 0.0])
mode_3_consts = np.array([0.0, 0.8, 0.0])
mode_4_consts = np.array([0.0, 0.8, 0.0])

# Bundle up all the matrices and constants
mode_mats = [mode_1_mat, mode_2_mat, mode_3_mat, mode_4_mat]
mode_consts = [mode_1_consts, mode_2_consts, mode_3_consts, mode_4_consts]

# Probabilities of obtaining each mode
mode_probabilities = np.array([0.1, 0.6, 0.15, 0.15])

In [ ]:
# Perform the simulation
for i in range(1, num_points):
    mode = np.random.choice([0, 1, 2, 3], p=mode_probabilities)

    points[i, :] = mode_mats[mode] @ points[i - 1, :] + mode_consts[mode]

In [ ]:
# Plot simulation results
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

# The ordering of the x,y,z points were chosen to get the best
# default view.
ax.scatter(ys=points[:, 0], zs=points[:, 1], xs=points[:, 2], s=0.5);

## Ballistic

periodic bcs

In [ ]:
# Number of particles to simulate
num_particles = 25000

# Number sites on ring substrate
num_substrate_sites = 300

# Array of heights at each substrate site
heights = np.zeros(num_substrate_sites)

# Array of particle positions (once deposited)
points = np.zeros((num_particles, 2))

In [ ]:
# Functions to deal with boundary conditions
left_site = lambda site: site - 1
right_site = lambda site: (site + 1) % num_substrate_sites

In [ ]:
# Perform the simulation
for i in range(num_particles):
    site = np.random.randint(num_substrate_sites)

    # Determine the maximum height of the neighbouring sites
    neighbour_max_height = np.array(
        [heights[s] for s in [left_site(site), right_site(site)]]
    ).max()

    # Determine where to place the particle
    if heights[site] >= neighbour_max_height:
        heights[site] += 1
    else:
        heights[site] = neighbour_max_height

    points[i, :] = np.array([site, heights[site]])

In [ ]:
# Plot simulation results
_, ax = plt.subplots()

ax.scatter(x=points[:, 0], y=points[:, 1], s=1.5)

# Cosmetics
ax.set_xlim([0, num_substrate_sites])
ax.set_ylim(ymin=0)

ax.set_xlabel("substrate site")
ax.set_ylabel("height")
ax.set_aspect("equal")

## Correlated

In [ ]:
# Number of particles to simulate
num_particles = 25000

# Number sites on ring substrate
num_substrate_sites = 300

# Array of heights at each substrate site
heights = np.zeros(num_substrate_sites)

# Array of particle positions (once deposited)
points = np.zeros((num_particles, 2))

In [ ]:
# Functions to deal with boundary conditions
left_site = lambda site: site - 1
right_site = lambda site: (site + 1) % num_substrate_sites

In [ ]:
# Perform the simulation
i = 0

while i < num_particles:
    site = np.random.randint(num_substrate_sites)

    # Determine the maximum height of the neighbouring sites
    neighbour_max_height = np.array(
        [heights[s] for s in [left_site(site), right_site(site)]]
    ).max()

    # Determine where to place the particle
    if heights[site] >= neighbour_max_height:
        candidate_height = heights[site] + 1
    else:
        candidate_height = neighbour_max_height

    candidate_point = np.array([site, candidate_height])

    # Determine whether to deposit the particle
    # based on its distance from the previous particle
    if i == 0:
        # First particle always gets placed
        pass
    else:
        # Calculate distance from previous particle. We need
        # to be careful with periodic boundary conditions here.
        dx = abs(points[i - 1, 0] - site)
        mindx = min(dx, num_substrate_sites - dx,)
        dsquared = mindx ** 2 + (points[i - 1, 1] - candidate_height) ** 2

        if np.random.random() < 5 / dsquared:
            pass
        else:
            continue

    # Deposit the particle
    heights[site] = candidate_height
    points[i, :] = candidate_point

    i += 1

In [ ]:
# Plot simulation results
_, ax = plt.subplots()

ax.scatter(x=points[:, 0], y=points[:, 1], s=1.5)

# Cosmetics
ax.set_xlim([0, num_substrate_sites])
ax.set_ylim(ymin=0)

ax.set_xlabel("substrate site")
ax.set_ylabel("height")
ax.set_aspect("equal")

# Discussion

Write discussion here